In [ ]:
# Import Libraries
import os
from dotenv import load_dotenv

from genai.extensions.langchain import LangChainInterface
from genai.schemas import GenerateParams
from genai.credentials import Credentials
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts.prompt import PromptTemplate
load_dotenv()
# Get the API key and URL from the environment variables
api_key = os.getenv("GENAI_KEY", None)
api_url = os.getenv("GENAI_API", None)
# Create a Credentials object to pass to the LangChainInterface
creds = Credentials(api_key, api_endpoint=api_url)
# Create a GenerateParams object to pass to the LangChainInterface
params = GenerateParams(
            decoding_method="greedy",
            max_new_tokens=500,
            min_new_tokens=50,
            temperature=0.7,
            stop_sequences=["Friend:"],
        )
# Create a LangChainInterface object to use for generating text
llm = LangChainInterface(model="meta-llama/llama-2-70b-chat", params=params, credentials=creds)

# Now we can override it and set it to "Friend"
template = """You will be helping for funrniture company by responding customer questions. Like personal preferences for the customers. The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know. If you not find any information about the questions answer generally by your own knowledge.

Current conversation:
{history}
Friend: {input}
AI:"""

#template=template.format(context=context, history="", input="")

PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)
conversation = ConversationChain(
    prompt=PROMPT,
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory(human_prefix="Friend"),
)
# query = "Do you prefer second-hand or new furniture??"
# conversation.predict(input=query)


In [ ]:
# Audio Summary Generation using watsonX Part - 2
## 1. Importing Libraries
import os
from dotenv import load_dotenv
import os

from genai.extensions.langchain import LangChainInterface
from genai.schemas import GenerateParams
from genai.credentials import Credentials

from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain import PromptTemplate
from langchain.document_loaders import TextLoader

## 4. Function to store open the txt file, convert it in to embeddings and store it in vector Database
def load_txt(): 
    pdf_name = 'transcriptios.txt'
    loaders = [TextLoader(pdf_name)]

    index = VectorstoreIndexCreator(
        embedding = HuggingFaceEmbeddings(model_name='all-MiniLM-L12-v2'), 
        text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    ).from_loaders(loaders)

    return index
## 5. Create the object of the function and call it
index = load_txt()
## 6. Deffining the PROMPT TEMPLATE
prompt_template = """You are a furniture persnalization assistant. Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. If you not find any information about the questions answer generally by your own knowledge.

{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
## 7. Defing the chain with parameters
chain_type_kwargs = {"prompt": PROMPT}

qa_chain = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=index.vectorstore.as_retriever(), chain_type_kwargs=chain_type_kwargs)

conversation_chain = ConversationChain(
    prompt=PROMPT,
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory(human_prefix="Friend"),
)


# ## 8. Test time
# query = "Do you prefer second-hand or new furniture??"

# print(qa.run(query))


In [ ]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(
    chains=[qa, conversation], verbose=True
)

review = overall_chain.run("Do you prefer second-hand or new furniture?")